<a href="https://colab.research.google.com/github/danm91/le_kingmakers/blob/testing/willdamann_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("mnt")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g5hCkB41svqmnH35AaBbKiBCfVelm-I6FWUNWRt_A75Ky1bnt9Q1UM
Mounted at mnt


In [3]:
%cd "mnt/My Drive/Colab Notebooks"

/content/mnt/My Drive/Colab Notebooks


In [4]:
import pandas as pd
import re
import string
import pickle
from nltk import tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from kingmakers.preproc_text import process_tweets
from kingmakers.preproc_abbv import abbreviations, convert_abbrev_in_text
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import xgboost as xgb

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [5]:
df = pd.read_csv('kingmakers/training.1600000.processed.noemoticon.csv', encoding='latin', names = ['sentiment','id','date','query','user','tweet'])

In [6]:
df_test = df.sample(frac = 0.05, random_state=0)[['sentiment', 'tweet']]

In [ ]:
preprocess = Pipeline([
    ('preprocess', TextPreprocess()),
    ('abbreviation', TextConvertAbbv())])


In [ ]:
df_test_2 = df_test

In [ ]:
df_test_2['tweet'] = preprocess.transform(df_test_2['tweet'])

In [ ]:
df_test_2

,sentiment,tweet
557138,0,want compete want hard competition want rally ...
349381,0,seems stuck ground amarillo put ground stop fl...
182051,0,pinking shear rarararrrarararrbabyproofing cut...
571236,0,0ff t0 meetin hate people v0lunteer free timegrrr
1339637,4,reply pls
...,...,...
1350300,4,think get raise mommy salary not cavity right ...
651945,0,heading dmv soon quotlegallyquot ride amp avoi...
390068,0,discouraged lack time management
1058149,4,good call sukses dong selalu buat kamu


In [ ]:
df_test

,sentiment,tweet
557138,0,want compete want hard competition want rally ...
349381,0,seems stuck ground amarillo put ground stop fl...
182051,0,pinking shear rarararrrarararrbabyproofing cut...
571236,0,0ff t0 meetin hate people v0lunteer free timegrrr
1339637,4,reply pls
...,...,...
1350300,4,think get raise mommy salary not cavity right ...
651945,0,heading dmv soon quotlegallyquot ride amp avoi...
390068,0,discouraged lack time management
1058149,4,good call sukses dong selalu buat kamu


In [31]:
df_test_prep = df_test
df_test_prep['tweet'] = df_test_prep['tweet'].apply(lambda x: process_tweets(x))
df_test_prep['tweet'] = df_test_prep['tweet'].apply(lambda x: convert_abbrev_in_text(x))


# Count Vectorizer

In [10]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer



In [86]:
X=df_test_prep['tweet']
y=df_test_prep['sentiment']
X_train1, X_test, y_train1, y_test = train_test_split(X, y, test_size=0.20,random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train1, y_train1, test_size = 0.2, random_state=0)

In [87]:

X_train_token = [word_tokenize(i) for i in X_train]


X_test_token = [word_tokenize(i) for i in X_val]

139181     good newsand fridayeven better yeah laugh shal...
844880                                     thank going watch
1042124    ah see got hopeful determined part going rest ...
796702                                            miss skank
667179                                 drop toilet like wife
                                 ...                        
386679     stuck car park docklands manage finish dinner ...
811423                  hi nice emeet pretty good hows thing
467604     really need sleep seeing not spell right ahhh ...
223189                                             next year
871289                 awww great way start day old daughter
Name: tweet, Length: 51200, dtype: object

In [105]:
import gensim.downloader as api
word2vec_transfer = api.load('glove-twitter-50')

KeyboardInterrupt: ignored

In [88]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [106]:
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train_token)
X_test_embed_2 = embedding(word2vec_transfer, X_test_token)

In [107]:
maxlen = len(X_train_embed_2[0])
for i in X_train_embed_2:
  if len(i) > maxlen:
    maxlen = len(i) 
                         

In [108]:
maxlen

28

In [102]:
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post')
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post')

In [121]:
y_train = y_train.map(lambda x: 1 if x==4 else 0)

In [122]:
y_train

139181     0
844880     1
1042124    1
796702     0
667179     0
          ..
386679     0
811423     1
467604     0
223189     0
871289     1
Name: sentiment, Length: 51200, dtype: int64

In [130]:
from tensorflow.keras.layers import Bidirectional

In [137]:
def glove_lstm():
    model = Sequential()
    
    model.add(layers.Masking())
    
    model.add(Bidirectional(layers.LSTM(
        28, 
        return_sequences = True, 
        recurrent_dropout=0.2
    )))
    
    model.add(layers.GlobalMaxPool1D())
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(28, activation = "relu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(28, activation = "relu"))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation = 'sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

model1 = glove_lstm()


In [124]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=50, restore_best_weights=True)

In [125]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers
def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(64, activation='tanh'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model
      

model = init_model()

In [126]:
model.fit(X_train_pad_2, y_train, 
          batch_size = 64,
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/100
560/560 [==============================] - 13s 17ms/step - loss: 0.5567 - accuracy: 0.7127 - val_loss: 0.5290 - val_accuracy: 0.7307
Epoch 2/100
560/560 [==============================] - 8s 15ms/step - loss: 0.5143 - accuracy: 0.7437 - val_loss: 0.5176 - val_accuracy: 0.7393
Epoch 3/100
560/560 [==============================] - 8s 14ms/step - loss: 0.4924 - accuracy: 0.7583 - val_loss: 0.5142 - val_accuracy: 0.7480
Epoch 4/100
560/560 [==============================] - 8s 15ms/step - loss: 0.4763 - accuracy: 0.7693 - val_loss: 0.5069 - val_accuracy: 0.7473
Epoch 5/100
560/560 [==============================] - 8s 15ms/step - loss: 0.4636 - accuracy: 0.7773 - val_loss: 0.4905 - val_accuracy: 0.7597
Epoch 6/100
560/560 [==============================] - 8s 15ms/step - loss: 0.4531 - accuracy: 0.7835 - val_loss: 0.4918 - val_accuracy: 0.7573
Epoch 7/100
560/560 [==============================] - 8s 15ms/step - loss: 0.4409 - accuracy: 0.7916 - val_loss: 0.4887 - val_accuracy

KeyboardInterrupt: ignored

In [ ]:
model1.fit(X_train_pad_2, y_train, 
          batch_size = 64,
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/100
560/560 [==============================] - 49s 81ms/step - loss: 0.6941 - accuracy: 0.6008 - val_loss: 0.5866 - val_accuracy: 0.7029
Epoch 2/100
560/560 [==============================] - 45s 81ms/step - loss: 0.5879 - accuracy: 0.6999 - val_loss: 0.5428 - val_accuracy: 0.7276
Epoch 3/100
560/560 [==============================] - 45s 81ms/step - loss: 0.5575 - accuracy: 0.7259 - val_loss: 0.5228 - val_accuracy: 0.7432
Epoch 4/100
560/560 [==============================] - 45s 81ms/step - loss: 0.5426 - accuracy: 0.7379 - val_loss: 0.5108 - val_accuracy: 0.7477
Epoch 5/100
560/560 [==============================] - 45s 81ms/step - loss: 0.5343 - accuracy: 0.7443 - val_loss: 0.5010 - val_accuracy: 0.7544
Epoch 6/100
560/560 [==============================] - 45s 81ms/step - loss: 0.5244 - accuracy: 0.7486 - val_loss: 0.4970 - val_accuracy: 0.7556
Epoch 7/100
560/560 [==============================] - 45s 80ms/step - loss: 0.5210 - accuracy: 0.7549 - val_loss: 0.4964 - val_ac